In [52]:
import pandas as pd
import os
import torch
import numpy as np
from torch.utils.data import Dataset
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [53]:
data_path = '../data/' 

location_path = os.path.join(data_path, 'sdwpf_baidukddcup2022_turb_location.CSV')
location      = pd.read_csv(location_path)
location.describe()

,TurbID,x,y
count,134.000000,134.000000,134.000000
mean,67.500000,2666.760881,5986.126609
std,38.826537,1829.842484,3346.503906
min,1.000000,0.000000,0.000000
25%,34.250000,1014.935725,3182.294825
50%,67.500000,3246.541850,5841.894340
75%,100.750000,4320.609625,8814.832950
max,134.000000,5501.452900,12121.004260


In [54]:
df_train = pd.read_csv(os.path.join(data_path, 'wtbdata_245days.csv')) 
print(df_train.shape)
print(df_train.columns)
df_train.head()

(4727520, 13)
Index(['TurbID', 'Day', 'Tmstamp', 'Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir',
       'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv'],
      dtype='object')


,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,1,1,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,00:10,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66
2,1,1,00:20,6.27,-2.18,30.60,41.63,20.91,1.0,1.0,1.0,-0.24,509.76
3,1,1,00:30,6.42,-0.73,30.52,41.52,20.91,1.0,1.0,1.0,-0.26,542.53
4,1,1,00:40,6.25,0.89,30.49,41.38,20.91,1.0,1.0,1.0,-0.23,509.36


In [55]:
df_train = pd.merge(df_train, location, on='TurbID', how='left')
print(df_train.shape)
print(df_train.columns)
df_train.head()

(4727520, 15)
Index(['TurbID', 'Day', 'Tmstamp', 'Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir',
       'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv', 'x', 'y'],
      dtype='object')


,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv,x,y
0,1,1,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3349.8515,5939.23193
1,1,1,00:10,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66,3349.8515,5939.23193
2,1,1,00:20,6.27,-2.18,30.60,41.63,20.91,1.0,1.0,1.0,-0.24,509.76,3349.8515,5939.23193
3,1,1,00:30,6.42,-0.73,30.52,41.52,20.91,1.0,1.0,1.0,-0.26,542.53,3349.8515,5939.23193
4,1,1,00:40,6.25,0.89,30.49,41.38,20.91,1.0,1.0,1.0,-0.23,509.36,3349.8515,5939.23193


In [56]:
df_train.describe()

,TurbID,Day,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv,x,y
count,4.727520e+06,4.727520e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.678002e+06,4.727520e+06,4.727520e+06
mean,6.750000e+01,1.230000e+02,5.028376e+00,4.975428e-01,4.110668e+01,2.739717e+01,1.885826e+02,2.685400e+01,2.683916e+01,2.682679e+01,-1.323900e+01,3.504458e+02,2.666761e+03,5.986127e+03
std,3.868139e+01,7.072483e+01,3.393703e+00,3.160275e+01,8.529011e+01,1.832832e+01,1.632459e+02,3.883566e+01,3.882940e+01,3.882235e+01,7.044244e+01,4.249932e+02,1.823002e+03,3.333994e+03
min,1.000000e+00,1.000000e+00,0.000000e+00,-3.030460e+03,-2.730300e+02,-2.731700e+02,-8.848600e+02,-1.000000e+01,-1.000000e+01,-1.000000e+01,-6.249800e+02,-9.330000e+00,0.000000e+00,0.000000e+00
25%,3.400000e+01,6.200000e+01,2.240000e+00,-3.830000e+00,1.405000e+01,2.006000e+01,6.302000e+01,1.000000e-02,1.000000e-02,1.000000e-02,-3.861000e+01,-3.000000e-01,1.014317e+03,3.134452e+03
50%,6.750000e+01,1.230000e+02,4.340000e+00,-2.300000e-01,2.573000e+01,3.008000e+01,1.947900e+02,5.400000e-01,5.400000e-01,5.400000e-01,-3.000000e-01,1.790800e+02,3.246542e+03,5.841894e+03
75%,1.010000e+02,1.840000e+02,7.010000e+00,3.420000e+00,3.273000e+01,3.682000e+01,3.219600e+02,8.399000e+01,8.398000e+01,8.398000e+01,-3.000000e-02,5.476600e+02,4.320683e+03,8.816238e+03
max,1.340000e+02,2.450000e+02,2.629000e+01,2.266950e+03,3.943300e+02,3.242100e+02,7.006200e+02,9.998000e+01,9.998000e+01,9.998000e+01,4.852000e+02,1.567020e+03,5.501453e+03,1.212100e+04


In [57]:
df_train.groupby(['Day'])['Wspd'].count()

Day
1      19151
2      19280
3      19280
4      19282
5      19281
       ...  
241    19284
242    19270
243    19283
244    19285
245    19283
Name: Wspd, Length: 245, dtype: int64

In [58]:
# df_train['time'] = pd.to_datetime(df_train['Tmstamp'], format='%H:%M')
# df_train['minute'] = df_train['time'].dt.minute
# df_train.head()

In [59]:
# df_train.fillna(method='ffill', inplace=True)

print(f'Wspd缺失比例：', df_train['Wspd'].isna().sum() / len(df_train))
df_train.isna().sum()

Wspd缺失比例： 0.010474413646055437


TurbID         0
Day            0
Tmstamp        0
Wspd       49518
Wdir       49518
Etmp       49518
Itmp       49518
Ndir       49518
Pab1       49518
Pab2       49518
Pab3       49518
Prtv       49518
Patv       49518
x              0
y              0
dtype: int64

In [60]:
print(df_train.columns)
print(df_train.select_dtypes(include='number').columns)

Index(['TurbID', 'Day', 'Tmstamp', 'Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir',
       'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv', 'x', 'y'],
      dtype='object')
Index(['TurbID', 'Day', 'Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2',
       'Pab3', 'Prtv', 'Patv', 'x', 'y'],
      dtype='object')


In [50]:
def fill_missing(df, groupby_label='TurbID', method='mean'):
    """
    缺失值填补. 
    """
    continuous_cols  = df.select_dtypes(include='number').columns
    catogorical_cols = df.select_dtypes(exclude='number').columns
    print(f'连续变量有：{continuous_cols}\n'
          f'非连续变量有： {catogorical_cols}')
    
    for col in tqdm(df.columns):
        if col in continuous_cols:
            # 用每个风机该变量的均值进行缺失值填充
            df[col].fillna(df.groupby(groupby_label)[col].transform(method), inplace=True)
        else:
            # 用每个风机类别变量的众数进行缺失值填充
            df[col].fillna(df.groupby(groupby_label)[col].transform(lambda x: x.mode()[0]), inplace=True)

    print(f'缺失值个数： {df.isna().sum().sum()}')

    return df
# df_train = fill_missing(df_train)

连续变量有：Index(['TurbID', 'Day', 'Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2',
       'Pab3', 'Prtv', 'Patv', 'x', 'y'],
      dtype='object')
非连续变量有： Index(['Tmstamp'], dtype='object')


100%|██████████| 15/15 [00:01<00:00,  8.32it/s]


缺失值个数： 0


In [61]:
print(f'缺失值个数： {df_train.isna().sum().sum()}')
df_train.fillna(method='bfill', inplace=True)
print(f'缺失值个数： {df_train.isna().sum().sum()}')
df_train.head()

缺失值个数： 495180
缺失值个数： 0


,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv,x,y
0,1,1,00:00,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66,3349.8515,5939.23193
1,1,1,00:10,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66,3349.8515,5939.23193
2,1,1,00:20,6.27,-2.18,30.60,41.63,20.91,1.0,1.0,1.0,-0.24,509.76,3349.8515,5939.23193
3,1,1,00:30,6.42,-0.73,30.52,41.52,20.91,1.0,1.0,1.0,-0.26,542.53,3349.8515,5939.23193
4,1,1,00:40,6.25,0.89,30.49,41.38,20.91,1.0,1.0,1.0,-0.23,509.36,3349.8515,5939.23193


In [62]:
df_train.isna().sum()

TurbID     0
Day        0
Tmstamp    0
Wspd       0
Wdir       0
Etmp       0
Itmp       0
Ndir       0
Pab1       0
Pab2       0
Pab3       0
Prtv       0
Patv       0
x          0
y          0
dtype: int64

## 逐个构建训练集

In [7]:
df_train['TurbID'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134], dtype=int64)

In [ ]:
class WindDataset_single(Dataset):
    def __init__(self, data,  
                 label_y,
                 window_len,
                 scaler='minmax',
                 ):
        super().__init__()
        self.data       = data
        self.label_y    = label_y
        self.window_len = window_len
        if scaler.lower() == 'minmax':
            self.scaler = MinMaxScaler()
            # self.scaler_x = MinMaxScaler()
            # self.scaler_y = MinMaxScaler()
        elif scaler.lower() == 'standard':
            self.scaler = StandardScaler()
            # self.scaler_x = StandardScaler()
            # self.scaler_y = StandardScaler()
        else:
            self.scaler = None

        self._prepare_data()
        
    def __len__(self):
        return len(self.data)

    def _prepare_data(self):
        self.data.replace(np.nan, value=0, inplace=True)

        if self.scaler:
            self.data = self.scaler.fit_transform(self.data)
        else:
            self.data = self.data.values

        

        
        
        

    def get_train_data(self, index, to_tensor=False):
        df_train.replace(np.nan, value=0, inplace=True)
        X_train = df_train.drop(columns=[self.label_y])
        Y_train = df_train[self.label_y]
        
        if self.scaler:
            X_train = self.scaler_x.fit_transform(X_train)
            Y_train = self.scaler_y.fit_transform(Y_train)

        # 按 TurbID 分组并对每个 TurbID 进行时间序列划分
        sequences = []
        for _, group in df_train.groupby('TurbID'):
            # 按时间戳排序
            group = group.sort_values(by='Tmstamp')

            # 选择适当的历史窗口和预测窗口大小
            history_window_size = 10
            prediction_window_size = 1

            for i in range(len(group) - history_window_size - prediction_window_size + 1):
                history = group.iloc[i:i+history_window_size][['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2', 'Pab3', 'Prtv', 'x', 'y']].values
                target = group.iloc[i+history_window_size:i+history_window_size+prediction_window_size]['Patv'].values
                sequences.append((torch.tensor(history, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)))



In [68]:
df_train.head()
print(len(df_train['Day'].unique()))
df_train['Day'].unique()

245


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [64]:
class Scaler(object):
    def __init__(self):
        self.mean = 0.
        self.std = 1.

    def fit(self, data):
        self.mean = np.mean(data)
        self.std = np.std(data)

    def transform(self, data):
        mean = torch.tensor(self.mean).to(data.device) if torch.is_tensor(data) else self.mean
        std = torch.tensor(self.std).to(data.device) if torch.is_tensor(data) else self.std
        return (data - mean) / std

    def inverse_transform(self, data):
        mean = torch.tensor(self.mean) if torch.is_tensor(data) else self.mean
        std = torch.tensor(self.std) if torch.is_tensor(data) else self.std
        return (data * std) + mean

class WindTurbineDataset(Dataset):
    def __init__(self, data_path, 
                 filename='my.csv', 
                 flag='train', 
                 size=None, 
                 turbine_id=0, 
                 task='MS', 
                 target='Target',
                 scale=True, 
                 start_col=2, 
                 day_len=24 * 6, 
                 train_days=15, 
                 val_days=3, 
                 test_days=6, 
                 total_days=30):
        super().__init__()
        self.unit_size = day_len
        if size is None:
            self.input_len = self.unit_size
            self.output_len = self.unit_size
        else:
            self.input_len = size[0]
            self.output_len = size[1]
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.task = task
        self.target = target
        self.scale = scale
        self.start_col = start_col
        self.data_path = data_path
        self.filename = filename
        self.tid = turbine_id
        self.total_size = self.unit_size * total_days
        self.train_size = train_days * self.unit_size
        self.val_size = val_days * self.unit_size
        self.test_size = test_days * self.unit_size
        self.__read_data__()

    def __read_data__(self):
        self.scaler = Scaler()
        df_raw = pd.read_csv(os.path.join(self.data_path, self.filename))
        border1s = [self.tid * self.total_size,
                    self.tid * self.total_size + self.train_size - self.input_len,
                    self.tid * self.total_size + self.train_size + self.val_size - self.input_len
                    ]
        border2s = [self.tid * self.total_size + self.train_size,
                    self.tid * self.total_size + self.train_size + self.val_size,
                    self.tid * self.total_size + self.train_size + self.val_size + self.test_size
                    ]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        df_data = df_raw
        if self.task == 'M':
            cols_data = df_raw.columns[self.start_col:]
            df_data = df_raw[cols_data]
        elif self.task == 'MS':
            cols_data = df_raw.columns[self.start_col:]
            df_data = df_raw[cols_data]
        elif self.task == 'S':
            df_data = df_raw[[self.tid, self.target]]

        pd.set_option('mode.chained_assignment', None)
        df_data.replace(to_replace=np.nan, value=0, inplace=True)

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values
        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        self.raw_data = df_data[border1 + self.input_len:border2]

    def __getitem__(self, index):
        if self.set_type >= 3:
            index = index * self.output_len
        s_begin = index
        s_end = s_begin + self.input_len
        r_begin = s_end
        r_end = r_begin + self.output_len
        seq_x = torch.from_numpy(self.data_x[s_begin:s_end])
        seq_y = torch.from_numpy(self.data_y[r_begin:r_end])
        return seq_x, seq_y

    def __len__(self):
        if self.set_type < 3:
            return len(self.data_x) - self.input_len - self.output_len + 1
        return int((len(self.data_x) - self.input_len) / self.output_len)

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [70]:
data_path  = '../data'
filename   = 'wtbdata_245days.csv'
flag       = 'train'
input_len  = 144
output_len = 288
size       = [input_len, output_len]
task       = 'MS'
target     = 'Patv'
start_col  = 3
turbine_id = 0
day_len    = 144
train_days = 153
val_days   = 16
test_days  = 15
total_days = 184

data_set = WindTurbineDataset(
    data_path  = data_path,
    filename   = filename,
    flag       = flag,
    size       = size,
    task       = task,
    target     = target,
    start_col  = start_col,
    turbine_id = turbine_id,
    day_len    = day_len,
    train_days = train_days,
    val_days   = val_days,
    test_days  = test_days,
    total_days = total_days
)

In [78]:
x, y = data_set[1]
x

tensor([[-0.2886, -0.3321, -0.1833,  ..., -0.3107, -0.3161,  1.8054],
        [-0.2881, -0.3244, -0.1839,  ..., -0.3107, -0.3161,  1.8702],
        [-0.2875, -0.3182, -0.1842,  ..., -0.3107, -0.3161,  2.0106],
        ...,
        [-0.2611, -0.3077, -0.1929,  ..., -0.3104, -0.3159,  6.3278],
        [-0.2724, -0.3168, -0.1937,  ..., -0.3107, -0.3159,  5.5233],
        [-0.2694, -0.3218, -0.1946,  ..., -0.3107, -0.3162,  5.9788]],
       dtype=torch.float64)

In [77]:
data_set.inverse_transform(y)


tensor([[ 7.0000e+00,  8.4000e-01,  2.7870e+01,  ...,  1.0000e+00,
         -1.6000e-01,  7.1812e+02],
        [ 5.8700e+00, -2.6100e+00,  2.7380e+01,  ...,  9.9000e-01,
         -2.7000e-01,  4.4215e+02],
        [ 4.0000e+00,  3.2600e+00,  2.7130e+01,  ...,  9.9000e-01,
         -2.1000e-01,  1.9604e+02],
        ...,
        [ 1.2410e+01, -8.1300e+00,  2.4950e+01,  ...,  1.0700e+00,
         -1.9000e-01,  1.5400e+03],
        [ 1.2000e+01, -2.8300e+00,  2.4880e+01,  ...,  1.0600e+00,
         -2.1000e-01,  1.5414e+03],
        [ 1.0350e+01, -1.4000e-01,  2.4750e+01,  ...,  1.0100e+00,
         -2.7000e-01,  1.4874e+03]], dtype=torch.float64)